In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline
!pip install timm
!pip install tqdm

In [ ]:
import os
import shutil
import tempfile
import time
import matplotlib.pyplot as plt
import PIL
import random
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import StepLR
import monai
from monai.apps import DecathlonDataset
from monai.config import print_config
from monai.data import DataLoader, decollate_batch,ImageDataset,Dataset
from monai.handlers.utils import from_engine
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric,ROCAUCMetric
from monai.networks.nets import SegResNet,DenseNet121
from monai.optimizers import LearningRateFinder
from monai.transforms import (
    Compose,
    LoadImaged,
    Orientationd,
    Resized,
    RandRotate90d,
    CropForegroundd,
    ScaleIntensityd,
    Activations, 
    AsDiscrete,
    EnsureChannelFirstd,
    Rand3DElasticd,
    RandRotated,
    PadListDataCollate,
    RandScaleIntensityd,
    SpatialPadd,
    RandAffined,
    RandFlipd,
    Spacingd,
    RandAdjustContrastd,
)
from monai.utils import set_determinism
import timm 
from torchvision.models import swin_transformer
from __future__ import annotations

import re
from collections import OrderedDict
from collections.abc import Callable, Sequence

import torch
from torch.hub import load_state_dict_from_url

from monai.networks.layers.factories import Conv, Dropout, Pool
from monai.networks.layers.utils import get_act_layer, get_norm_layer
from monai.utils.module import look_up_option
from tqdm import tqdm
from sklearn.metrics import f1_score, recall_score
import torch.nn as nn
import torch.nn.functional as F
import math
import SimpleITK as sitk

In [ ]:
pin_memory = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

PATH_DATASET = '/kaggle/input/ucsf-seg/segImgs'
#df_train = pd.read_csv(('/kaggle/input/ucsf-csv/UCSF-MGMT.csv'),usecols=['ID', 'MGMT'])
df_train = pd.read_csv(('/kaggle/input/ucsf-csv/UCSF-os-train_stratified.csv'),usecols=['ID', 'MGMT','IDH','age','gender','WHO CNS Grade','OS','EOR','fold'])
images = []
weight_decay=0.01
auc_metric = ROCAUCMetric()

In [ ]:
import torch
import torch.nn as nn
from typing import Tuple
import torch.nn.functional as F  # 添加：用于 F.gelu


try:
    from monai.networks.nets import SwinUNETR
   
    try:
        from timm.models.layers import DropPath
    except ImportError:
        from monai.networks.blocks.patchembedding import DropPath
    MONAI_AVAILABLE = True
except ImportError:
    print("Warning: MONAI or its components (SwinUNETR, DropPath) not found. Using dummy classes.")
    MONAI_AVAILABLE = False
    class SwinUNETR:
        def __init__(self, *args, **kwargs): pass
    class DropPath(nn.Module):
        def __init__(self, *args, **kwargs): super().__init__()
        def forward(self, x): return x

class LayerNormChannelsFirst(nn.Module):
    def __init__(self, normalized_shape, eps=1e-6):
        super().__init__()
       

    def forward(self, x):
        
        return x
    
class ConvNeXtBlock3D(nn.Module):
    def __init__(self, dim, layer_scale_init_value=1e-6, drop_path=0.):
       
    def forward(self, x):

        return x

# --- CrossAttentionFusionBlock (不变) ---
class CrossAttentionFusionBlock(nn.Module):
   
    def __init__(self, embed_dim, num_heads, dropout=0.1):
        super().__init__()

    def forward(self, x_t1c, x_flair):
       
        return fused_map



class HyCoSNet(nn.Module):
    def __init__(
        self,
        in_channels: int = 1,
        out_channels: int = 2,  
        swin_feature_size: int = 24,
        use_checkpoint: bool = True
    ):
        super().__init__()
        if not MONAI_AVAILABLE:
            raise RuntimeError("MONAI is not available, cannot create HyCoSNet.")

        self.flair_encoder = SwinUNETR(
            in_channels=in_channels, out_channels=out_channels,
            feature_size=swin_feature_size, use_checkpoint=use_checkpoint, spatial_dims=3
        )

       
        self.t1c_stem = nn.Sequential(
            nn.Conv3d(in_channels, convnext_dims[0], kernel_size=4, stride=4),
            LayerNormChannelsFirst(normalized_shape=convnext_dims[0])
        )
        self.t1c_stage1 = ConvNeXtBlock3D(dim=convnext_dims[0])
        

    def forward(self, mri_t1c, mri_flair, extra_features=None):
        
        return logits  # (B, 2)

In [ ]:
import os
import pandas as pd

def readIMGs(imgPath, csvFile, imgtype, N):
    train_files = [] # 确保列表已初始化

    if N == 0:
        tfolds = [1, 2, 3, 4]
    elif N == 1:
        tfolds = [0, 2, 3, 4]
    elif N == 2:
        tfolds = [0, 1, 3, 4]
    elif N == 3:
        tfolds = [0, 1, 2, 4]
    elif N == 4:
        tfolds = [0, 1, 2, 3]
    
    for _, row in csvFile.iterrows():
        id_ = row['ID']
        if int(row['fold']) in tfolds:
          
            id_modified = id_[0:10] + '0' + id_[10:]
            img1_path = os.path.join(imgPath, id_modified + imgtype[0] + "_trans.nii")
            img2_path = os.path.join(imgPath, id_modified + imgtype[1] + "_trans.nii")

            # 2. 检查这两个文件路径是否都真实存在
            if os.path.exists(img1_path) and os.path.exists(img2_path):
                # 3. 如果都存在，则添加信息到列表中
                train_files.append({
                    "img1": img1_path,
                    "img2": img2_path,
                    "age": row['age'],
                    "gender": row['gender'],
                
                    "label": row['OS'], 
                })
            else:
                # （可选）可以打印更详细的警告
                if not os.path.exists(img1_path):
                    print(f"Warning: Image not found for {img1_path}")
                if not os.path.exists(img2_path):
                    print(f"Warning: Image not found for {img2_path}")

   
    return train_files

def readIMGsTest(imgPath, csvFile, imgtype, foldN):
    test_files = [] 
    
    for _, row in csvFile.iterrows():
        id_ = row['ID']
        if int(row['fold']) == foldN:
            id_modified = id_[0:10] + '0' + id_[10:]
  
            img1_path = os.path.join(imgPath, id_modified + imgtype[0] + "_trans.nii")
            img2_path = os.path.join(imgPath, id_modified + imgtype[1] + "_trans.nii")
            
            # 检查两个文件是否存在
            if os.path.exists(img1_path) and os.path.exists(img2_path):
                test_files.append({
                    "img1": img1_path,
                    "img2": img2_path,
                   
                    "age": row['age'],
                    "gender": row['gender'],
                   
                    "label": row['OS'],
                })
    
    # 返回正确的变量名
    return test_files

In [ ]:
def transformers():
    train_transforms = Compose(
    [
      
        LoadImaged(keys=["img1", "img2"]),
        EnsureChannelFirstd(keys=["img1", "img2"]),
        Orientationd(keys=["img1", "img2"], axcodes="RAS"),
        CropForegroundd(keys=["img1", "img2"], source_key="img1", allow_smaller=True),
        
       
        RandFlipd(keys=["img1", "img2"], prob=0.3, spatial_axis=0),  
        RandFlipd(keys=["img1", "img2"], prob=0.3, spatial_axis=1),  
        RandRotate90d(keys=["img1", "img2"], prob=0.2, spatial_axes=(0, 2)),  
        RandAffined(
            keys=["img1", "img2"],
            prob=0.2, 
            rotate_range=(np.pi / 72, np.pi / 72, np.pi / 72),  
            scale_range=(0.05, 0.05, 0.05),  
            mode=("bilinear", "bilinear"),
            padding_mode="zeros"
        ),
        
       
        Resized(keys=["img1", "img2"], spatial_size=(130, 170, 130)),
        SpatialPadd(keys=["img1", "img2"], spatial_size=(130, 170, 130)), 
        
      
        ScaleIntensityd(keys=["img1", "img2"]),
        RandScaleIntensityd(keys=["img1", "img2"], factors=0.05, prob=0.3), 
        RandAdjustContrastd(keys=["img1", "img2"], prob=0.3, gamma=(0.95, 1.05)), 
       
    ])
    val_transforms = Compose(
    [
        LoadImaged(keys=["img1","img2"]),
        EnsureChannelFirstd(keys=["img1","img2"]),
        ScaleIntensityd(keys=["img1","img2"]),
        Orientationd(keys=["img1","img2"],axcodes="RAS"),
        CropForegroundd(keys=["img1","img2"],source_key="img1",allow_smaller=True),
        Resized(keys=["img1","img2"], spatial_size=(130,170,130)),
    ])
    return train_transforms,val_transforms

In [ ]:
def calculate_balanced_weights(num_samples_class0, num_samples_class1, smoothing_factor=0.1):
    
    total_samples = num_samples_class0 + num_samples_class1
    
    # 使用sqrt来减少极端类别不平衡的影响
    weight_0 = torch.sqrt(torch.tensor(total_samples / (num_samples_class0 + smoothing_factor)))
    weight_1 = torch.sqrt(torch.tensor(total_samples / (num_samples_class1 + smoothing_factor)))
    
    weights = torch.tensor([weight_0, weight_1])
    
    # 归一化权重到合理范围
    min_weight = weights.min()
    max_weight = weights.max()
    weights = (weights - min_weight) / (max_weight - min_weight + 1e-6)
    weights = weights * 0.9 + 0.1  # 确保最小权重为0.1
    
    return weights

In [ ]:
import torch.nn.functional as F  # for one-hot

def custom_collate(batch):
    collated = {}
    collated['img1'] = torch.stack([item['img1'] for item in batch])
    collated['img2'] = torch.stack([item['img2'] for item in batch])
    
    # --- FIX IS HERE ---
    # The labels must be of type torch.long for CrossEntropyLoss
    collated['label'] = torch.tensor([item['label'] for item in batch], dtype=torch.long)
    
    # extra: [age, gender_onehot(2), grade_onehot(3), IDH, MGMT, EOR] -> (B, 8)
    ages = torch.tensor([item['age'] for item in batch], dtype=torch.float).unsqueeze(1)  # (B, 1)
    genders = torch.tensor([item['gender'] for item in batch], dtype=torch.long)  # 假设 0=男, 1=女
    gender_onehot = F.one_hot(genders, num_classes=2).float()  # (B, 2)
    collated['extra'] = torch.cat([ages, gender_onehot], dim=1)  # (B, 1+2+3 = 6)
    return collated

In [ ]:

def train_fold(fold, trainlist, val_list, train_transforms, val_transforms, device, pin_memory):

    train_ds = Dataset(data=trainlist, transform=train_transforms)
    train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=4, pin_memory=pin_memory, collate_fn=custom_collate)
    val_ds = Dataset(data=val_list, transform=val_transforms)
    val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=1, pin_memory=pin_memory, collate_fn=custom_collate)

   
    model = HyCoSNet(
        in_channels=1,
        out_channels=2,  
        swin_feature_size=24,
        use_checkpoint=True
    ).to(device)
    
 
    train_labels = [data['label'] for data in trainlist]
    num_class_0 = train_labels.count(0)
    num_class_1 = train_labels.count(1)
    if num_class_0 == 0 or num_class_1 == 0:
        class_weights = None
    else:
        w0 = (num_class_0 + num_class_1) / (2.0 * num_class_0)
        w1 = (num_class_0 + num_class_1) / (2.0 * num_class_1)
        class_weights = torch.tensor([w0, w1], dtype=torch.float32).to(device)

   
    weight_decay = 0.05
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=weight_decay)
    loss_function = torch.nn.CrossEntropyLoss(weight=class_weights)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)
    
   
    scaler = torch.amp.GradScaler('cuda')  # 修复 AMP 警告
    accumulation_steps = 4

   
    max_epochs = 50
    val_interval = 1
    best_auc = -1
    epoch_loss_values = []
    auc_metric = ROCAUCMetric()
    early_stop_patience = 20
    early_stop_counter = 0
    
    for epoch in range(max_epochs):
        epoch_loss = 0
        model.train()
        optimizer.zero_grad()
        
        for i, batch_data in tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Fold {fold} Epoch {epoch + 1}"):
            inputs0 = batch_data['img1'].to(device, non_blocking=True)
            inputs1 = batch_data['img2'].to(device, non_blocking=True)
            labels = batch_data['label'].to(device, non_blocking=True)
            extra = batch_data['extra'].to(device, non_blocking=True)

            with torch.amp.autocast('cuda'):  
                outputs = model(inputs0, inputs1, extra)
                loss = loss_function(outputs, labels)
                loss = loss / accumulation_steps

            scaler.scale(loss).backward()
            epoch_loss += loss.item() * accumulation_steps

            if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

        epoch_loss /= len(train_loader)
        epoch_loss_values.append(epoch_loss)
        
        if (epoch + 1) % val_interval == 0:
            model.eval()
            y_pred_list = []
            y_list = []
            
            with torch.no_grad():
                with torch.amp.autocast('cuda', dtype=torch.float16):
                    for val_data in val_loader:
                        val_image0 = val_data["img1"].to(device, non_blocking=True)
                        val_image1 = val_data["img2"].to(device, non_blocking=True)
                        val_labels = val_data["label"].to(device, non_blocking=True)
                        val_extra = val_data['extra'].to(device, non_blocking=True)
                        outputs = model(val_image0, val_image1, val_extra)
                        y_pred_list.append(outputs)
                        y_list.append(val_labels)
            
            y_pred = torch.cat(y_pred_list, dim=0)
            y = torch.cat(y_list, dim=0)

            y_pred_cpu = y_pred.cpu()
            y_cpu = y.cpu()

            post_pred = Compose([Activations(softmax=True)])
            y_pred_processed = post_pred(y_pred_cpu)

            y_cpu_with_channel = y_cpu.unsqueeze(1)
            post_label = Compose([AsDiscrete(to_onehot=2, dim=1)])
            y_true_processed = post_label(y_cpu_with_channel)
            
            auc_metric(y_pred_processed, y_true_processed)
            auc_result = auc_metric.aggregate()
            auc_metric.reset()

            y_pred_class = y_pred_cpu.argmax(dim=1).numpy()
            y_true_class = y_cpu.numpy()

            acc = (y_pred_class == y_true_class).mean()
            f1 = f1_score(y_true_class, y_pred_class, average='binary', zero_division=0)
            recall = recall_score(y_true_class, y_pred_class, average='binary', zero_division=0)
            
            print(f"Fold {fold} Epoch {epoch + 1}/{max_epochs} | Loss: {epoch_loss:.4f} | Val AUC: {auc_result:.4f} | Acc: {acc:.4f} | F1: {f1:.4f} | Recall: {recall:.4f}")
            
            scheduler.step()

            if auc_result > best_auc:
                best_auc = auc_result
                early_stop_counter = 0
                torch.save(model.state_dict(), f"/kaggle/working/best_model_fold_{fold}.pth")
                print(f"--> New best model saved with AUC: {best_auc:.4f}")
            else:
                early_stop_counter += 1
                print(f"AUC did not improve. Early stopping counter: {early_stop_counter}/{early_stop_patience}")

            if early_stop_counter >= early_stop_patience:
                print(f"--- Early stopping triggered at epoch {epoch + 1} ---")
                break
    return best_auc


def main():
    # Clear GPU memory
    torch.cuda.empty_cache()

    # Paths and initialization
    from monai.utils import set_determinism
    set_determinism(seed=42)
    imgtype = ('_T1c_bias', '_FLAIR_bias')
    pin_memory = True
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
 
    fold_aucs = []
    for fold in [4]:
        print(f"\nStarting fold {fold}")
        trainlist = readIMGs(PATH_DATASET, df_train, imgtype, fold)
        val_list = readIMGsTest(PATH_DATASET, df_train, imgtype, fold)
        
   
        age_values = np.array([data['age'] for data in trainlist])
        age_mean = age_values.mean()
        age_std = age_values.std() + 1e-8
        for data in trainlist + val_list:
            data['age'] = (data['age'] - age_mean) / age_std

 
        print(f"Train samples: {len(trainlist)}, Val samples: {len(val_list)}")
        if len(val_list) < 2:
            print(f"Warning: Validation set too small for fold {fold}, AUC may be unreliable")
        else:
            val_labels = [data['label'] for data in val_list]
            print(f"Val label distribution: 0={val_labels.count(0)}, 1={val_labels.count(1)}")

        train_transforms, val_transforms = transformers()
        best_auc = train_fold(fold, trainlist, val_list, train_transforms, val_transforms, device, pin_memory)
        fold_aucs.append(best_auc)
    
    print(f"\nFolds [0, 1] results: AUC = {fold_aucs}")
    print(f"Mean AUC: {np.mean(fold_aucs):.4f} ± {np.std(fold_aucs):.4f}")

if __name__ == "__main__":
    main()